# Translating a Landlab non-raster grid into PyVista for visualization

*Greg Tucker, CU Boulder, June 2025*

PyVista offers powerful 3D visualization capabilities for geoscientific data. This tutorial shows how to translate a non-raster Landlab grid into a pair of PyVista objects that can be viewed interactively as 3D surfaces.

The `llpytools` package offers a function called `grid_to_pv` that translates the contents of a Landlab grid and its fields into (generally) two PyVista mesh objects, one for each of the two dual meshes that compose most Landlab grids. In one of the resulting PyVista meshes, the points are Landlab grid *nodes* and the cells are Landlab grid *patches*. In the other, the points are Landlab grid *corners* and the cells are Landlab grid *cells*.

## The dual nature of Landlab grids

Most Landlab grid types contain two complementary or "dual" meshes. One mesh is built around a set of points called *nodes*. Adjacent pairs of nodes are joined by directed line segments called *links*. Nodes form the vertices for a set of polygons called *patches*. In a `RasterModelGrid`, for example, the nodes are arranged in a regular, rectilinear pattern; its patches are rectangles, and the links are the edges of these rectangles. In a `HexModelGrid`, the nodes are the vertices of a set of equilateral triangles. (For illustrations and examples, see Hobley et al. (2017)).

In addition to the mesh of nodes-links-patches, a Landlab grid (other than the special `NetworkModelGrid`) contains a dual mesh. The elements of this dual mesh are called, in Landlab terminology, corners, faces, and cells. In a `RasterModelGrid`, this dual is itself a regular rectilinear grid. In a grid that uses a Delaunay triangulation for nodes, the dual mesh is a Voronoi diagram. This reflects the fact that the Delaunay triangular and Voronoi tesselation are dual to one another: each Vornoi polygon represents the area of space that is closer to the node within it than to any other node in the grid, and the vertices of the Voronoi polygons are the circumcenters of the Delaunay triangles.

Part of the motivation for this dual-mesh model is to support the implementation of finite-difference and finite-volume numerical methods on Landlab grids. Finite-difference methods need to have discrete points at which solutions are tracked; finite-volume methods (in 2d) require polygons. Having a dual mesh allows one to have both of these: every *cell* in a Landlab grid has a *node* inside; all nodes except those around the perimeter sit inside cells. For more, see Hobley et al. (2017) and Barnhart et al. (2020). Many Landlab components use a combination of nodes, links, and cells; it is also possible, though less widely done to date, to use corners, faces, and patches.

## A note on terminology

Landlab and PyVista use somewhat different terminology for grid elements. In PyVista, a *cell* is a 2d or 3d geometric object. In Landlab, *cell* refers to a polygon in one of a grid's two complementary meshes. In this tutorial, the two will generally be distinguished by referring specifically to a "Landlab cell" or "PyVista cell".

In [ ]:
import numpy as np
import pyvista as pv
from llpvtools import grid_to_pv
from landlab import (
    IcosphereGlobalGrid,
    NetworkModelGrid,
    RadialModelGrid,
    imshow_grid,
)
from landlab.plot.graph import plot_graph

## A tiny example

To illustrate how Landlab's data structures translate into PyVista's, it is useful to start with a tiny example. Here we create a Landlab `RadialModelGrid`, a type of grid that uses a Delaunay triangulation to connect *nodes* into a mesh of triangular *patches*, and a corresponding Voronoi diagram that defines the Landlab *cells* and their vertices, which in Landlab-speak are called *corners*.

This mesh will have 2 *rings*, meaning that the grid has a central node plus two "rings" of nodes. We will ask Landlab to put 5 nodes in the innermost (first) ring. This arrangement will create a node-link-patch mesh with 16 nodes, 35 links, and 20 triangular patches. The dual mesh will have 19 nodes, 25 faces, and 6 cells. One of the 6 cells will be a pentagon; the others will be hexagons: a useful arrangement if we want to demonstrate translation of a Landlab grid that has polygons with differing numbers of sides.

Here we create the grid and add two fields: one at nodes, and one at corners.

In [ ]:
rmg = RadialModelGrid(n_rings=2, nodes_in_first_ring=5)
zn = rmg.add_zeros("z_at_node", at="node")
zn[:] = 2.0 - np.sqrt(rmg.x_of_node**2 + rmg.y_of_node**2)
zc = rmg.add_zeros("z_at_corner", at="corner")
zc[:] = 2.0 - np.sqrt(rmg.x_of_corner**2 + rmg.y_of_corner**2)

Here's what the data structure looks like:

In [ ]:
rmg

Landlab's `plot_graph()` function displays the mesh elements. By default, `plot_graph()` displays nodes, links, and patches, together with their ID numbers:

In [ ]:
plot_graph(rmg)

The `at` keyword allows us to specify the elements of the dual mesh: corners, faces, and cells:

In [ ]:
plot_graph(rmg, at="corner,face,cell")

## Translating to PyVista and plotting

The `llpvtools` function `grid_to_pv` translates a Landlab grid into two PyVista mesh objects, one for each of the two complementary Landlab meshes. The specific type of PyVista data object depends on the type of Landlab grid. In this next example, the type will be `UnstructuredGrid`.

In [ ]:
pv_node_mesh, pv_cnr_mesh = grid_to_pv(rmg, field_for_node_z=zn, field_for_corner_z=zc)
print(type(pv_node_mesh), type(pv_cnr_mesh))

Once we have a PyVista mesh object for each of the two complementary Landlab meshes, we can display them with the object's `plot` method:

In [ ]:
pv_node_mesh.plot(show_edges=True)

In [ ]:
pv_cnr_mesh.plot(show_edges=True)

### Rendering in 3D

Setting the optional `make3d` parameter to `True` will create 3D objects by adding a base layer of nodes and corners, respectively, to the node and corner meshes. 

The optional arguments `values_for_node_base` and `values_for_corner_base` allow you to set $z$ coordinate values for the base of the object, which can be a constant, an array equal in length to the number of nodes or corners, or the name of a node or corner field in the grid.

In [ ]:
pv_node_3d, pv_cnr_3d = grid_to_pv(
    rmg, field_for_node_z=zn, field_for_corner_z=zc, make3d=True
)

In [ ]:
pv_node_3d.plot(show_edges=True)

In [ ]:
pv_cnr_3d.plot(show_edges=True)

## Visualizing an `IcosphereGlobalGrid`

Landlab's `IcosphereGlobalGrid` creates a quasi-spherical grid using the same dual mesh approach as with other grid types. The `mesh_densification_level` parameter determines the resolution of the grid. If no densification is applied (which is the default), the two dual meshes consist of an icosahedron (twenty triangular sides, like a "d20" in D&D) and a dodecahedron (twelve hexagonal sides, like a "d12").

### Simple `IcosphereGlobalGrid` example: icosahedron and dodecahedron

Here's a visualization of these shapes using PyVista:

In [ ]:
ggrid = IcosphereGlobalGrid()

In [ ]:
pv_global_node_mesh, pv_global_cnr_mesh = grid_to_pv(ggrid)

In [ ]:
pv_global_node_mesh.plot(show_edges=True)

In [ ]:
pv_global_cnr_mesh.plot(show_edges=True)

If there are no fields in the grid, all of the polygons will have the same color. If we add one or more fields, we can use these to color the faces. Here we'll add a cell field and a node field with a simple 1-12 numbering:

In [ ]:
ggrid = IcosphereGlobalGrid()
c = np.arange(1, 13)
ggrid.add_field("node_value", c, at="node")
ggrid.add_field("cell_value", c, at="cell")
pv_global_node_mesh, pv_global_cnr_mesh = grid_to_pv(ggrid)

In [ ]:
pv_global_node_mesh.plot(show_edges=True)

Because the values are associated with nodes, PyVista will interpolate to create a gradational color across the triangular patches. By contrast, when values are associated with polygons---in this case, the Landlab grid's cells, a single uniform color will be applied across the polygon surface:

In [ ]:
pv_global_cnr_mesh.plot(show_edges=True)

### Example of global topography

In this example, we create an `IcosphereGlobalGrid` with a mesh densification level of five, representing a point spacing of about 240 km, and read in a global topography file with elevation values corresponding to that geometry:

In [ ]:
gtgrid = IcosphereGlobalGrid(radius=6372000.0, mesh_densification_level=5)
topo_data = np.loadtxt("global_elevation_etopo_ico_level5.txt")
topo = gtgrid.add_field("topographic__elevation", topo_data, at="node")
celltopo = gtgrid.add_field("topographic__elevation", topo_data, at="cell")
pv_global_node_mesh, pv_global_cnr_mesh = grid_to_pv(gtgrid)

In [ ]:
pv_global_cnr_mesh.plot()

By the way, if we want to check the spacing between grid points, we can take advantage of the `length_of_link` attribute:

In [ ]:
print(
    "The spacing between grid nodes is",
    np.round(np.mean(gtgrid.length_of_link) / 1000.0, 1),
    "plus or minus",
    np.round(np.std(gtgrid.length_of_link) / 1000.0, 1),
    "km",
)

## Visualizing a `NetworkModelGrid`

A `NetworkModelGrid` is a special type of Landlab grid that represents the nodes and links in a network. Unlike other grid types, a `NetworkModelGrid` *only* has nodes and links, and does not have patches, cells, corners, or faces.

When passed a `NetworkModelGrid`, the `grid_to_pv()` function returns only one rather than two PyVista meshes (the second return will be `None`). The return is a PyVista `UnstructuredGrid` object that contains lines instead of cells. Any node fields in the original Landlab grid will be added to the `UnstructuredGrid` as `point_data`, as usual. *Any link fields will be added as* `cell_data` (because PyVista considers the lines to be a form of cell) , so that links in a `NetworkModelGrid` can be colored according to any desired *link* field.

The example below creates, translates, and visualizes a tiny `NetworkModelGrid`:

In [ ]:
y_of_node = (0, 1, 2, 2, 3, 3)
x_of_node = (0, 0, -0.5, 0.5, 0, 1)
z_of_node = (0., 0.1, 0.4, 0.3, 0.6, 0.6)
nodes_at_link = ((0, 1), (2, 1), (1, 3), (3, 4), (3, 5))
nmg = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)
z = nmg.add_field("node_elevation", z_of_node, at="node")
order = nmg.add_field("stream_order", np.array([3, 2, 2, 1, 1]), at="link")

The Landlab `plot_graph()` function allows us to visualize the network structure and the numbering of nodes and links. (Note that the link directions do *not* indicate anything about flow direction; they just follow the Landlab convention that links "point" toward the upper-right half-space.)

In [ ]:
plot_graph(nmg, at="node,link")

Translate to a PyVista object and plot (leaving the second return blank since in this case there is only one):

In [ ]:
pv_nmg, _ = grid_to_pv(nmg, field_for_node_z=z)
pv_nmg.plot()

To visualize a link-based quantity, in this case the `stream_order` field that we created (representing Strahler stream order), we simply switch the active scalar:

In [ ]:
pv_nmg.set_active_scalars('stream_order')
pv_nmg.plot()